# Load the Data

In [1]:
import csv
from tqdm import tqdm
import numpy as np
from scipy.sparse import dok_matrix

ratings_file = '/home/boyto/DataSets/MovieLensResearch/ratings.csv'

#MAX_RATINGS = 20000263
MAX_RATINGS = 5000000
NUM_MOVIES = 131264
NUM_USERS = 138494

S = dok_matrix((NUM_MOVIES, NUM_USERS), dtype=np.float32)

i=0
max_user = 0
max_movie = 0
with open(ratings_file, 'rb') as csvfile:
    reader = csv.reader(csvfile)
    line1 = reader.next()
    for j in tqdm(range(0,MAX_RATINGS)):
        row = reader.next()
        user = int(row[0])
        movie = int(row[1])
        S[user,movie]=float(row[2])
        if user > max_user:
            max_user = user
        if movie > max_movie:
            max_movie = movie        
        if i>MAX_RATINGS:
            break
        i = i +1
print max_user,max_movie

100%|██████████| 5000000/5000000 [01:52<00:00, 44492.82it/s]

34395 131258


# Step 1: Create Lens (SVD)

In [48]:
from scipy.sparse.linalg import svds
[u,s,v] = svds(S, k=1, which='LM')

In [49]:
u.shape

(131264, 2)

In [50]:
u[1010,0]

-0.00052777183

In [53]:
import numpy as np
np.savetxt('/home/boyto/DataSets/MovieLensResearch/svd.csv', u)

In [54]:
u.max()

0.030035667

In [55]:
u.min()

-0.03971079

# Step 2: Group by Lens using Overlaping Segments

In [56]:
INTERVALS = 10  # Number of intervals, the smaller the more compressed the dataset will be
OVERLAP = 0.2 # Overlap between segments

In [58]:
lens = u
lens_range = lens.max()-lens.min()
segment = lens_range / ((INTERVALS - 1) - ((INTERVALS - 1) * OVERLAP) + 1);
segment

0.0085056654927207209

In [59]:
# Group the elements of the Lens in the INTERVALS


131264